In [1]:
import  numpy as np
import cv2
from skimage import feature
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, precision_score, recall_score

from sklearn.manifold import TSNE
from xgboost import XGBClassifier
import umap
import torch
from torchvision import transforms
from transformers import ViTFeatureExtractor

from skimage import feature
import torch
import torchvision.transforms as transforms
from torchvision.models import vision_transformer
import timm
import pandas as pd

#import nibabel as nib

C:\Users\vikra\.conda\envs\impana_py39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Function to extract HOG features from an image
def extract_hog_features(image):
    hog_features, _ = feature.hog(image, orientations=9, pixels_per_cell=(64, 64),
                                  cells_per_block=(3, 3), block_norm='L2-Hys', visualize=True)
    return hog_features

In [3]:
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes = 0)    
model = model.eval()

In [4]:
# Function to extract Vision Transformer features from an image
def extract_vision_transformer_features(image):
    # Define a transformation to match the pre-trained Vision Transformer model requirements
    image_rgb = np.stack((image,) * 3, axis=-1)
    #print(image_rgb.shape)
    image_rgb = (image_rgb * 255).astype(np.uint8)
    # Define a transformation to match the pre-trained Vision Transformer model requirements
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    image_rgb = transform(image_rgb).unsqueeze(0)

   

    # Extract features using the pre-trained model
    with torch.no_grad():
        
        features = model(image_rgb)
    # Reshape the features
    features = features.squeeze().numpy()

    return features



In [20]:
data = pd.read_csv("meta_info.csv")

In [21]:
data

,patient_id,nodule_no,slice_no,original_image,mask_image,malignancy,is_cancer,is_clean
0,1,0,0,LIDC-IDRI-0001/0001_NI000_slice000,LIDC-IDRI-0001/0001_MA000_slice000,5,True,False
1,1,0,1,LIDC-IDRI-0001/0001_NI000_slice001,LIDC-IDRI-0001/0001_MA000_slice001,5,True,False
2,1,0,2,LIDC-IDRI-0001/0001_NI000_slice002,LIDC-IDRI-0001/0001_MA000_slice002,5,True,False
3,1,0,3,LIDC-IDRI-0001/0001_NI000_slice003,LIDC-IDRI-0001/0001_MA000_slice003,5,True,False
4,1,0,4,LIDC-IDRI-0001/0001_NI000_slice004,LIDC-IDRI-0001/0001_MA000_slice004,5,True,False
...,...,...,...,...,...,...,...,...
16885,1011,2,2,LIDC-IDRI-1011/1011_NI002_slice002,LIDC-IDRI-1011/1011_MA002_slice002,5,True,False
16886,1011,2,3,LIDC-IDRI-1011/1011_NI002_slice003,LIDC-IDRI-1011/1011_MA002_slice003,5,True,False
16887,1011,2,4,LIDC-IDRI-1011/1011_NI002_slice004,LIDC-IDRI-1011/1011_MA002_slice004,5,True,False
16888,1012,0,1,LIDC-IDRI-1012/1012_NI000_slice001,LIDC-IDRI-1012/1012_MA000_slice001,2,False,False


In [22]:
import os
available_imgs = os.listdir("data/Image/")
def fun(filename):
    #print(f"Filename: {filename}")
    #print(f"Patient ID: {filename.split('/')[0]}")
    if filename.split("/")[0] in available_imgs:
        return True
    return False

def fun2(filename):
    #print(f"Filename: {filename}")
    #print(f"Patient ID: {filename.split('/')[0]}")
    return filename.split("/")[0] 

In [23]:
available_imgs

['.gitkeep',
 '.ipynb_checkpoints',
 'LIDC-IDRI-0001',
 'LIDC-IDRI-0002',
 'LIDC-IDRI-0003',
 'LIDC-IDRI-0004',
 'LIDC-IDRI-0005',
 'LIDC-IDRI-0006',
 'LIDC-IDRI-0007',
 'LIDC-IDRI-0008',
 'LIDC-IDRI-0009',
 'LIDC-IDRI-0010',
 'LIDC-IDRI-0011',
 'LIDC-IDRI-0012',
 'LIDC-IDRI-0013',
 'LIDC-IDRI-0014',
 'LIDC-IDRI-0015',
 'LIDC-IDRI-0016',
 'LIDC-IDRI-0017',
 'LIDC-IDRI-0018',
 'LIDC-IDRI-0019',
 'LIDC-IDRI-0020',
 'LIDC-IDRI-0021',
 'LIDC-IDRI-0022',
 'LIDC-IDRI-0023',
 'LIDC-IDRI-0024',
 'LIDC-IDRI-0025',
 'LIDC-IDRI-0026',
 'LIDC-IDRI-0027',
 'LIDC-IDRI-0028',
 'LIDC-IDRI-0029',
 'LIDC-IDRI-0030',
 'LIDC-IDRI-0031',
 'LIDC-IDRI-0032',
 'LIDC-IDRI-0033',
 'LIDC-IDRI-0034',
 'LIDC-IDRI-0035',
 'LIDC-IDRI-0036',
 'LIDC-IDRI-0037',
 'LIDC-IDRI-0038',
 'LIDC-IDRI-0039',
 'LIDC-IDRI-0040',
 'LIDC-IDRI-0041',
 'LIDC-IDRI-0042',
 'LIDC-IDRI-0043',
 'LIDC-IDRI-0044',
 'LIDC-IDRI-0045',
 'LIDC-IDRI-0046',
 'LIDC-IDRI-0047',
 'LIDC-IDRI-0048',
 'LIDC-IDRI-0049',
 'LIDC-IDRI-0050',
 'LIDC-IDRI-00

In [14]:
data['original_image'].apply(fun2).unique()#[:30]#.isin(available_imgs)

array(['LIDC-IDRI-0028', 'LIDC-IDRI-0032', 'LIDC-IDRI-0062',
       'LIDC-IDRI-0071', 'LIDC-IDRI-0100', 'LIDC-IDRI-0143',
       'LIDC-IDRI-0174', 'LIDC-IDRI-0189', 'LIDC-IDRI-0197',
       'LIDC-IDRI-0205', 'LIDC-IDRI-0214', 'LIDC-IDRI-0218',
       'LIDC-IDRI-0224', 'LIDC-IDRI-0225', 'LIDC-IDRI-0226',
       'LIDC-IDRI-0239', 'LIDC-IDRI-0253', 'LIDC-IDRI-0261',
       'LIDC-IDRI-0279', 'LIDC-IDRI-0293', 'LIDC-IDRI-0295',
       'LIDC-IDRI-0306', 'LIDC-IDRI-0307', 'LIDC-IDRI-0316',
       'LIDC-IDRI-0322', 'LIDC-IDRI-0327', 'LIDC-IDRI-0330',
       'LIDC-IDRI-0331', 'LIDC-IDRI-0333', 'LIDC-IDRI-0336',
       'LIDC-IDRI-0342', 'LIDC-IDRI-0349', 'LIDC-IDRI-0361',
       'LIDC-IDRI-0364', 'LIDC-IDRI-0382', 'LIDC-IDRI-0383',
       'LIDC-IDRI-0389', 'LIDC-IDRI-0391', 'LIDC-IDRI-0401',
       'LIDC-IDRI-0410', 'LIDC-IDRI-0417', 'LIDC-IDRI-0418',
       'LIDC-IDRI-0422', 'LIDC-IDRI-0425', 'LIDC-IDRI-0428',
       'LIDC-IDRI-0441', 'LIDC-IDRI-0446', 'LIDC-IDRI-0455',
       'LIDC-IDRI-0465',

In [24]:
data = data[(data['original_image'].apply(fun))]# & data['data_split'] == 'Train']

In [25]:
data

,patient_id,nodule_no,slice_no,original_image,mask_image,malignancy,is_cancer,is_clean
0,1,0,0,LIDC-IDRI-0001/0001_NI000_slice000,LIDC-IDRI-0001/0001_MA000_slice000,5,True,False
1,1,0,1,LIDC-IDRI-0001/0001_NI000_slice001,LIDC-IDRI-0001/0001_MA000_slice001,5,True,False
2,1,0,2,LIDC-IDRI-0001/0001_NI000_slice002,LIDC-IDRI-0001/0001_MA000_slice002,5,True,False
3,1,0,3,LIDC-IDRI-0001/0001_NI000_slice003,LIDC-IDRI-0001/0001_MA000_slice003,5,True,False
4,1,0,4,LIDC-IDRI-0001/0001_NI000_slice004,LIDC-IDRI-0001/0001_MA000_slice004,5,True,False
...,...,...,...,...,...,...,...,...
16885,1011,2,2,LIDC-IDRI-1011/1011_NI002_slice002,LIDC-IDRI-1011/1011_MA002_slice002,5,True,False
16886,1011,2,3,LIDC-IDRI-1011/1011_NI002_slice003,LIDC-IDRI-1011/1011_MA002_slice003,5,True,False
16887,1011,2,4,LIDC-IDRI-1011/1011_NI002_slice004,LIDC-IDRI-1011/1011_MA002_slice004,5,True,False
16888,1012,0,1,LIDC-IDRI-1012/1012_NI000_slice001,LIDC-IDRI-1012/1012_MA000_slice001,2,False,False


In [26]:
hog_feats = []
vit_feats = []
labels = []

count = 0
for i, row in data.iterrows():
    #print(i)
    try:
        img = np.load("data/Image/"+row['original_image']+".npy")
    except Exception as e:
        print(f"Exception: {str(e)}")
        continue
    hog_feature = extract_hog_features(img)
    vit_feature = extract_vision_transformer_features(img)
    hog_feats.append(hog_feature)
    vit_feats.append(vit_feature)
    labels.append(int(row['is_cancer'] == 'True'))
    count += 1
    if count % 10 == 0:
        print(count)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
Exception: [Errno 2] No such file or directory: 'data/Image/LIDC-IDRI-0028/0028_CN001_slice000.npy'
Exception: [Errno 2] No such file or directory: 'data/Image/LIDC-IDRI-0028/0028_CN001_slice001.npy'
Exception: [Errno 2] No such file or directory: 'data/Image/LIDC-IDRI-0028/0028_CN001_slice002.npy'
Exception: [Errno 2] No such file or directory: 'data/Image/LIDC-IDRI-0028/0028_CN001_slice003.npy'
Exception: [Errno 2] No such file or directory: 'data/Image/LIDC-IDRI-0028/0028_CN001_slice004.npy'
Exception: [Errno 2] No such file or directory: 'data/Image/LIDC-IDRI-0028/0028_CN001_slice005.npy'
Exception: [Errno 2] No such file or directory: 'data/Image/LIDC-IDRI-0028/0028_CN001_slice006.npy'
Exception: [Errno 2] No such file or directory: 'data/Image/LIDC-IDRI-0028/0028_CN001_slice007.npy'
Exception: [Errno 2] No such file or directory: 'data/Image/LIDC-IDRI-0028/0028_CN001_slice00

In [27]:
import umap

In [28]:
print(f"Hog feats shape:{hog_feats[0].shape}")

Hog feats shape:(2916,)


In [29]:
print(f"Vit feats shape:{vit_feats[0].shape}")

Vit feats shape:(768,)


In [30]:
all_features = np.concatenate((np.stack(hog_feats), np.stack(vit_feats)), axis=1)
all_features.shape

(9919, 3684)

In [36]:
umap_model = umap.UMAP(n_components=50)
umap_features = umap_model.fit_transform(all_features)


In [37]:
X_train, X_test,y_train, y_test = train_test_split(umap_features, labels, test_size=0.2)#, random_state=0)

In [38]:
np.mean(y_test)

0.4561491935483871

In [39]:
# Train XGBoost classifier
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [40]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, precision_score, recall_score
y_pred = xgb_model.predict(X_train)
y_proba = xgb_model.predict_proba(X_train)[:, 1]

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_train, y_train)

# Calculate sensitivity and specificity
sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])
specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])

# Calculate AUC
auc = roc_auc_score(y_train, y_proba)

# Calculate precision
precision = precision_score(y_train, y_pred)
accuracy = accuracy_score(y_train, y_pred)


print(f"Train Accuracy: {accuracy}")
print(f"Train Sensitivity: {sensitivity}")
print(f"Train Specificity: {specificity}")
print(f"Train AUC: {auc}")
print(f"Train Precision: {precision}")

y_pred = xgb_model.predict(X_test)
y_proba = xgb_model.predict_proba(X_test)[:, 1]

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Calculate sensitivity and specificity
sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])
specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])

# Calculate AUC
auc = roc_auc_score(y_test, y_proba)

# Calculate precision
precision = precision_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("\n---------------\n")
print(f"Test Accuracy: {accuracy}")
print(f"Test Sensitivity: {sensitivity}")
print(f"Test Specificity: {specificity}")
print(f"Test AUC: {auc}")
print(f"Test Precision: {precision}")

Train Accuracy: 0.982356647763075
Train Sensitivity: 1.0
Train Specificity: 1.0
Train AUC: 0.9986108324171157
Train Precision: 0.9781042128603105

---------------

Test Accuracy: 0.8629032258064516
Test Sensitivity: 0.8265193370165745
Test Specificity: 0.8934198331788693
Test AUC: 0.939535276678324
Test Precision: 0.8667439165701043


In [45]:

from sklearn.model_selection import GridSearchCV

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'n_estimators': [100, 200, 300]
}

# Create an XGBoost regressor
xgb_model = XGBClassifier()

# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=5, verbose = 4)

# Fit the model to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and the corresponding score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV 1/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8;, score=0.679 total time=   0.1s
[CV 2/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8;, score=0.690 total time=   0.0s
[CV 3/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8;, score=0.679 total time=   0.0s
[CV 4/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8;, score=0.657 total time=   0.0s
[CV 5/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.8;, score=0.647 total time=   0.0s
[CV 1/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9;, score=0.680 total time=   0.0s
[CV 2/5] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.9;, score=0.691 total time=   0.0s
[CV 3

In [46]:
# Print the best parameters and the corresponding score
print("Best Parameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

Best Parameters:  {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.9}
Best Score:  0.8674228103339635


In [47]:
# Train XGBoost classifier
xgb_model = XGBClassifier(colsample_bytree = 1.0, learning_rate = 0.2, max_depth = 5, n_estimators = 300, subsample = 0.9)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=300, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [48]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, precision_score, recall_score
y_pred = xgb_model.predict(X_train)
y_proba = xgb_model.predict_proba(X_train)[:, 1]

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_train, y_train)

# Calculate sensitivity and specificity
sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])
specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])

# Calculate AUC
auc = roc_auc_score(y_train, y_proba)

# Calculate precision
precision = precision_score(y_train, y_pred)
accuracy = accuracy_score(y_train, y_pred)


print(f"Train Accuracy: {accuracy}")
print(f"Train Sensitivity: {sensitivity}")
print(f"Train Specificity: {specificity}")
print(f"Train AUC: {auc}")
print(f"Train Precision: {precision}")

y_pred = xgb_model.predict(X_test)
y_proba = xgb_model.predict_proba(X_test)[:, 1]

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

# Calculate sensitivity and specificity
sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])
specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])

# Calculate AUC
auc = roc_auc_score(y_test, y_proba)

# Calculate precision
precision = precision_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("\n---------------\n")
print(f"Test Accuracy: {accuracy}")
print(f"Test Sensitivity: {sensitivity}")
print(f"Test Specificity: {specificity}")
print(f"Test AUC: {auc}")
print(f"Test Precision: {precision}")

Train Accuracy: 0.9910522999369881
Train Sensitivity: 1.0
Train Specificity: 1.0
Train AUC: 0.9994475127495825
Train Precision: 0.9888858016115588

---------------

Test Accuracy: 0.8684475806451613
Test Sensitivity: 0.8408839779005525
Test Specificity: 0.891566265060241
Test AUC: 0.9424277646070897
Test Precision: 0.8667425968109339
